In [68]:
%matplotlib inline

import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from operator import truediv
from nxpd import draw
from collections import Counter
import itertools
import random

import collaboratr
reload(collaboratr)

<module 'collaboratr' from 'collaboratr.pyc'>

In [22]:
BREAKOUT_REQ_RATIO = 2
BREAKOUT_REQ_FRACTION = 0.2
G = nx.DiGraph()

In [23]:
# Retrieve data from Google Sheet and parse using pandas dataframe
data = pd.read_csv("https://raw.githubusercontent.com/benelson/collaboratr/master/collaboratr.csv")
data = data.replace(np.nan,' ', regex=True)
users = data['1. What is your name?']
emails = data['2. What is your email?']
teach = data['3. What can you help others with?']
learn = data['4. What do you want to collaborate on or learn about?']

## Decision Tree

In [27]:
breakouts = collaboratr.find_breakouts(users,teach,learn,BREAKOUT_REQ_RATIO,BREAKOUT_REQ_FRACTION)

In [75]:
print 'looks like youll be having breakouts in '
for b in breakouts: print b

looks like youll be having breakouts in 
GAIA DRI
packaging (Python)
gaussian processes
autodiff


## Draw Overall Hack Graph

In [32]:
# Insert users in graphs
for u,e,l,t in zip(users, emails, learn, teach):
    collaboratr.insert_node(G,u, email=e, learn=l.split(';'), teach=t.split(';'))

In [33]:
participants = G.nodes(data=True)

In [37]:
assign = collaboratr.assign_users(G,participants,breakouts)

In [39]:
G_new = nx.DiGraph(bgcolor='white')
G_new.add_nodes_from(participants, style='filled', fillcolor='black', 
                     fontcolor='white', fontname='sans-serif', fontsize=24)
G_new.add_node('breakout', shape='rectangle', width=4, height=1, style='filled', fillcolor='black',
              fontcolor='white', fontname='sans-serif', fontsize=30)

In [40]:
for p in G.nodes():
    mappings = assign[p]
    for m in mappings.keys():
        G_new.add_edge(p, mappings[m], label=m, style='dotted', fontcolor='red', fontsize=20,
                      fontname='sans-serif')

In [41]:
draw(G_new, filename='Resources/0.svg')

'Resources/0.svg'

# Draw Individually Customized Graphs

In [57]:
participants = G_new.nodes()
connections = G_new.edges(data=True)

In [58]:
index = collaboratr.drawIndividualGraphs(participants,connections)

In [59]:
obj = {}
obj['index'] = index
obj['participants'] = [p.split(' ')[0] for p in participants]

# Creating a Web Page

In [60]:
TEMPLATE1 = collaboratr.createWebPage(obj)

In [61]:
with open('display.html', 'w') as file:
    file.write(TEMPLATE1.render(obj=obj))

# Breakout Visualizations

In [66]:
participants = G.nodes(data=True)
TEMPLATE2 = collaboratr.createBreakoutPage(participants,breakouts)

In [67]:
with open('breakouts.html', 'w') as file:
    file.write(TEMPLATE2.render(objs=objs))